In [12]:
import pandas as pd
import random
from tqdm import tqdm

In [13]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

from algorithms.data.Retina import RetinaModel

In [14]:
data_dir = '../data/seed/'
seed = pd.read_csv(data_dir + 'SEED_progression.csv')

n_chunks = 100
chunk_size = len(seed) // n_chunks

chunks = [seed.iloc[i:i + chunk_size] for i in range(0, len(seed), chunk_size)]

if len(chunks) > n_chunks:
    chunks[-2] = pd.concat([chunks[-2], chunks[-1]], ignore_index=True)
    chunks = chunks[:-1]

seed

,sno,time_between_visits,gender,age,SBP,DBP,HR,bmi,anti_ht,anti_chol,...,CUPVOLUME_R_2,ONHCENTER_X_R_2,ONHCENTER_Y_R_2,Comments_ONH_R_2,Exclusioncodings_ONH_R_2,Anyotherremarks_ONH_R_2,eye,func_progression,glauc_progression,Patient
0,MS26151,NaN,2,65.086927,162.500000,68.0,63.500000,24.532644,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,R,-1,-1,MS26151_R
1,MS21290,NaN,2,43.819302,132.500000,84.5,90.500000,26.304339,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,R,-1,-1,MS21290_R
2,MS11063,NaN,2,66.313484,144.500000,70.0,53.500000,29.015688,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,R,-1,-1,MS11063_R
3,MS26049,5.648186,1,43.241615,144.000000,85.5,98.000000,45.068871,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,R,-1,0,MS26049_R
4,MS13148,NaN,1,77.075975,118.000000,63.5,96.000000,19.147334,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,R,-1,-1,MS13148_R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20061,CS51759,3.181383,1,48.531143,155.500000,89.5,88.500000,22.320562,0.0,0.0,...,0.003,97.0,98.0,NaN,.,NaN,L,-1,0,CS51759_L
20062,CS50489,NaN,1,55.504449,145.000000,89.5,110.333333,15.312441,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,L,-1,-1,CS50489_L
20063,CS55300,6.428474,1,61.393566,122.500000,80.0,101.500000,22.968750,0.0,0.0,...,0.196,100.0,98.0,NaN,.,NaN,L,-1,0,CS55300_L
20064,CS51844,5.571526,2,55.857632,138.666667,84.0,87.000000,23.109639,0.0,0.0,...,0.372,103.0,94.0,NaN,.,NaN,L,-1,0,CS51844_L


In [15]:
keys = seed.keys()
# for key in keys:
#     print(key)

In [16]:
def compute_map(sbp, dbp):
    """Compute Mean Arterial Pressure (MAP)."""
    return dbp + (1/3) * (sbp - dbp)

for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    chunk = chunk.reset_index(drop=True)

    sbp_samples = chunk['bpsys_f_2']
    dbp_samples = chunk['bpdia_f_2']

    map_samples = []
    for j in range(len(sbp_samples)):
        map_samples.append(compute_map(sbp_samples[j], dbp_samples[j]))

    temp = pd.DataFrame({
        'Patient': chunk['Patient'],
        'IOP': chunk['IOP_2'],
        'MAP': map_samples,
        'SBP': sbp_samples,
        'DBP': dbp_samples,
        'HR': chunk['bppul_f_2']
    })

    rm = RetinaModel(
        icp='../config/Initial_wIOP.csv',
        fn=temp,
        ret_fn=0
    )
    res = rm.pipeline()
    res.to_csv(data_dir + f'SEED_enhanced_2/{i}.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

subject 0 has a missing value
subject 1 has a missing value
subject 2 has a missing value
subject 4 has a missing value
subject 5 has a missing value
subject 6 has a missing value
subject 8 has a missing value
subject 15 has a missing value
subject 26 has a missing value
subject 33 has a missing value
subject 34 has a missing value
subject 35 has a missing value
subject 37 has a missing value
subject 40 has a missing value
subject 42 has a missing value
subject 48 has a missing value
subject 50 has a missing value
subject 54 has a missing value
subject 59 has a missing value
subject 60 has a missing value
subject 63 has a missing value
subject 64 has a missing value
subject 65 has a missing value
subject 67 has a missing value
subject 68 has a missing value
subject 70 has a missing value
subject 81 has a missing value
subject 82 has a missing value
subject 85 has a missing value
subject 92 has a missing value
subject 97 has a missing value
subject 98 has a missing value
subject 99 has 

  1%|          | 1/100 [02:32<4:11:22, 152.35s/it]

subject 0 has a missing value
subject 1 has a missing value
subject 11 has a missing value
subject 12 has a missing value
subject 13 has a missing value
subject 19 has a missing value
subject 20 has a missing value
subject 21 has a missing value
subject 25 has a missing value
subject 28 has a missing value
subject 29 has a missing value
subject 31 has a missing value
subject 37 has a missing value
subject 40 has a missing value
subject 41 has a missing value
subject 42 has a missing value
subject 47 has a missing value
subject 60 has a missing value
subject 69 has a missing value
subject 78 has a missing value
subject 81 has a missing value
subject 91 has a missing value
subject 92 has a missing value
subject 101 has a missing value
subject 102 has a missing value
subject 108 has a missing value
subject 109 has a missing value
subject 116 has a missing value
subject 117 has a missing value
subject 118 has a missing value
subject 123 has a missing value
subject 126 has a missing value
s

  2%|▏         | 2/100 [05:09<4:13:03, 154.94s/it]

...

 98%|█████████▊| 98/100 [4:00:20<05:57, 178.81s/it]

subject 2 has a missing value
subject 5 has a missing value
subject 7 has a missing value
subject 10 has a missing value
subject 11 has a missing value
subject 14 has a missing value
subject 15 has a missing value
subject 21 has a missing value
subject 22 has a missing value
subject 24 has a missing value
subject 28 has a missing value
subject 40 has a missing value
subject 49 has a missing value
subject 51 has a missing value
subject 55 has a missing value
subject 58 has a missing value
subject 64 has a missing value
subject 66 has a missing value
subject 68 has a missing value
subject 76 has a missing value
subject 77 has a missing value
subject 79 has a missing value
subject 82 has a missing value
subject 87 has a missing value
subject 88 has a missing value
subject 89 has a missing value
subject 95 has a missing value
subject 98 has a missing value
subject 105 has a missing value
subject 110 has a missing value
subject 115 has a missing value
subject 124 has a missing value
subject

 99%|█████████▉| 99/100 [4:03:16<02:57, 177.84s/it]

subject 2 has a missing value
subject 4 has a missing value
subject 9 has a missing value
subject 12 has a missing value
subject 17 has a missing value
subject 22 has a missing value
subject 26 has a missing value
subject 27 has a missing value
subject 29 has a missing value
subject 33 has a missing value
subject 34 has a missing value
subject 36 has a missing value
subject 39 has a missing value
subject 41 has a missing value
subject 55 has a missing value
subject 56 has a missing value
subject 72 has a missing value
subject 75 has a missing value
subject 81 has a missing value
subject 87 has a missing value
subject 93 has a missing value
subject 97 has a missing value
subject 111 has a missing value
subject 112 has a missing value
subject 114 has a missing value
subject 118 has a missing value
subject 120 has a missing value
subject 123 has a missing value
subject 124 has a missing value
subject 127 has a missing value
subject 141 has a missing value
subject 149 has a missing value
s

100%|██████████| 100/100 [4:07:08<00:00, 148.29s/it]
